# Loading raw video

In [6]:
import numpy as np
import cv2
import os
import glob
import math
import imutils

In [20]:
labeled_videos = glob.glob('./labeled/*.hevc')
video_index = 4

cap = cv2.VideoCapture(labeled_videos[video_index])

frame_count = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if(ret == False):
        break
    else:
        cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_count += 1
    
cap.release()
cv2.destroyAllWindows()

print(frame_count)

1196


 # Visualising GT pitch and yaw (respectively in 2D array)

In [7]:
def draw_gaze(image_in, cam_pos, pitchyaw, length=40.0, thickness=2, color=(255, 0, 0)):
    """Draw gaze angle on given image with a given cam positions."""
    image_out = image_in
    if len(image_out.shape) == 2 or image_out.shape[2] == 1:
        image_out = cv2.cvtColor(image_out, cv2.COLOR_GRAY2BGR)
    dx = -length * np.sin(pitchyaw[1])
    dy = -length * np.sin(pitchyaw[0])
    cv2.arrowedLine(image_out, tuple(np.round(cam_pos).astype(np.int32)),
                   tuple(np.round([cam_pos[0] + dx, cam_pos[1] + dy]).astype(int)), color,
                   thickness, cv2.LINE_AA, tipLength=0.3)
    return image_out

In [16]:
def draw_orientation(frame, current_p_y):
    pitch = 'Pitch: ' + str(current_p_y[0])
    yaw = 'Yaw:  ' + str(current_p_y[1])
    cv2.putText(frame, pitch, (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, 2)
    cv2.putText(frame, yaw, (20,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, 2)
    
    if(not math.isnan(current_p_y[0])):
        draw_gaze(frame, (582, 845), current_p_y, length=3000, thickness = 15)
        draw_gaze(frame, (582, 845), (0,0), length=3000, thickness = 15, color = (255,255,255)) #reference, perfect calib
        
    return frame

In [9]:
def is_contour_bad(c):
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)
	# the contour is 'bad' if it is not a rectangle
	return not len(approx) == 4

# Real-time lane edge detection

In [25]:
labeled_videos = glob.glob('./labeled/*.hevc')
gt_pitch_yaw = glob.glob('./labeled/*.txt')
video_index = 0


pitch_yaw = []


with open(gt_pitch_yaw[video_index], 'r') as f:
    for line in f.readlines():
        pitch_yaw.append(float(x) for x in line.split(' '))

#frame size (874, 1164, 3)
cap = cv2.VideoCapture(labeled_videos[video_index])
frame_count = 0


while(cap.isOpened()):
    ret, frame = cap.read()
    
    
    # if(frame_count == 30):
    #     cv2.imwrite('sample_img.png', frame)
    
    try:
        current_p_y = list(pitch_yaw[frame_count])
        
        ### comment the following for normal video
        frame = frame[450:700, 400:850]
        frame = cv2.GaussianBlur(frame, (13,13), 2)
        frame = cv2.Canny(frame, 0, 20)
        
        # find contours in the image and initialize the mask that will be
        # used to remove the bad contours
        cnts = cv2.findContours(frame.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        mask = np.ones(frame.shape[:2], dtype="uint8") * 255
        # loop over the contours
        for c in cnts:
            # if the contour is bad, draw it on the mask
            if is_contour_bad(c):
                cv2.drawContours(mask, [c], -1, 0, -1)
        # remove the contours from the image and show the resulting images
        frame = cv2.bitwise_and(frame, frame, mask=mask)
        ##
    except:
        next
    
    if(ret == False):
        break
    else:
        plot_frame = draw_orientation(frame, current_p_y)
        cv2.imshow('frame',plot_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
    frame_count += 1
    
cap.release()
cv2.destroyAllWindows()

print(frame_count)

1200


# Following interesting points using Lucas Kanade optical flow and Shi-Tomasi feature tracking

In [65]:
import numpy as np
import cv2 as cv

cap = cv2.VideoCapture(labeled_videos[3])
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

#Shi-Tomasi
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k = cv.waitKey(1) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()

No frames grabbed!


# Detecting edge lanes on single img

In [80]:
raw_img_gr = cv2.imread('sample_img.png', cv2.COLOR_BGR2GRAY)

blurry_img = cv2.GaussianBlur(raw_img_gr, (5,5), 1)
edges = cv2.Canny(blurry_img, 0, 100)

cv2.imshow('img',edges)
cv2.waitKey(0)

-1